This is just a test notebook used alongside the prelim data for practice purposes.

In [1]:
import pandas as pd
import nltk
import numpy as np
import json
import re
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, VarianceThreshold, f_regression, SelectPercentile
from sklearn.grid_search import GridSearchCV

# NEWS = 0
# UPNEWS = 1

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
#json_data=open('./API-data/ps_news').read()
#news_comments = json.loads(json_data)

In [3]:
json_data=open('./API-data/ps_news_posts').read()
news_posts = json.loads(json_data)

In [4]:
#json_data=open('./API-data/ps_upnews').read()
#upnews_comments = json.loads(json_data)

In [5]:
json_data=open('./API-data/ps_upnews_posts').read()
upnews_posts = json.loads(json_data)

In [6]:
import nltk
words = set(nltk.corpus.words.words())



#" ".join(w for w in nltk.wordpunct_tokenize(sent) \
#         if w.lower() in words or not w.isalpha())

In [7]:
upnews_posts = upnews_posts[:5500]

upnews_headlines = [li['title'] for li in upnews_posts]

df_upnews_head = pd.DataFrame(upnews_headlines, columns=['headlines'])

df_upnews_head.drop_duplicates(inplace=True)

In [8]:
df_upnews_head['news'] = 1

df_upnews_head.shape

(4784, 2)

In [9]:
news_posts = news_posts[:5500]

news_headlines = [li['title'] for li in news_posts]

df_news_head = pd.DataFrame(news_headlines, columns=['headlines'])

df_news_head.drop_duplicates(inplace=True)

In [10]:
df_news_head['news'] = 0

In [11]:
df_news_head.head()

,headlines,news
0,Steve Bannon disinvited from New Yorker festiv...,0
1,Brazil's National Museum Fire: What It Means f...,0
2,"Democrats, Eyeing a Majority, Prepare an Inves...",0
3,None of them were Redditors.,0
4,Seahawks Owner Gives $100k To Help Republicans...,0


In [12]:
df_upnews_head.head()

,headlines,news
0,10 Nigerian Celebrities Who Have Been Sentence...,1
1,Philadelphia’s Homeless Are Finding New Hope T...,1
2,Kindness can change a life,1
3,Deaf toddler hears for the first time,1
4,It’s not empty anymore.,1


In [13]:
df = pd.concat((df_upnews_head, df_news_head), axis=0, ignore_index=True)

In [14]:
df.headlines = df.headlines.map(lambda x: re.sub('[^a-zA-Z0-9\s]','', x))
df.headlines = df.headlines.map(lambda x: re.sub('/r/News', ' ', x))
df.headlines = df.headlines.map(lambda x: re.sub('/r/Upliftingnews', ' ', x))
df.headlines = df.headlines.map(lambda x: re.sub('http[^\s]*', ' ', x))


In [15]:
df.tail(10)

,headlines,news
9946,Hire Expert Residential Painters in Mosman at ...,0
9947,vinay hari office raid by income tax departmen...,0
9948,Key Reasons to Purchase Mercedes through their...,0
9949,Bun chuyn gia nh thiu n 15 tui nhy cu t t,0
9950,Get the Best Quality and Long Lasting Whiteboa...,0
9951,Australia Plans to Deny Chelsea Manning an Ent...,0
9952,Cigar Humidor Cabinet,0
9953,Smashed 1987 Porsche 959 may get 500000 at auc...,0
9954,Best location in Udaipur for Weddings Wedding...,0
9955,Chemnitz protests Far right on march in east G...,0


In [16]:
target= df['news']

#df_upnews_head['word_count'] = df_upnews_head.headlines.map(lambda x: len(x.split()))

cvec = CountVectorizer(stop_words='english', min_df=10, max_df=.95)

term_mat = cvec.fit_transform(df['headlines'])

In [17]:
cvec.get_feature_names()[3330:]

[]

In [18]:
term_df = pd.DataFrame(term_mat.toarray(), columns=cvec.get_feature_names())

In [19]:
#term_df.insert(0, 'target', target)

# TF-IDF

In [20]:
tfidf = TfidfVectorizer(stop_words='english', min_df=10, max_df=.95, ngram_range=(1, 3))

In [21]:
term_mat = tfidf.fit_transform(df['headlines'])

In [22]:
len(tfidf.get_feature_names())

1755

In [23]:
term_df = pd.SparseDataFrame(term_mat, columns=tfidf.get_feature_names())

In [24]:
term_df.fillna(0, inplace=True)

In [25]:
SVD = TruncatedSVD(n_components=100)
svd_matrix = SVD.fit_transform(term_df)
svd_matrix.shape

(9956, 100)

In [26]:
component_names = ["component_"+str(i+1) for i in range(100)]
svd_df = pd.DataFrame(svd_matrix,
                      columns=component_names)

In [27]:
svd_df.head()

,component_1,component_2,component_3,component_4,component_5,component_6,component_7,component_8,component_9,component_10,...,component_91,component_92,component_93,component_94,component_95,component_96,component_97,component_98,component_99,component_100
0,4.378269e-04,3.581281e-02,4.190590e-02,6.067975e-03,-2.301837e-03,1.238206e-02,-1.407425e-02,-4.478882e-02,2.724372e-02,-6.869977e-02,...,9.332478e-03,9.965965e-03,1.414001e-03,3.585334e-02,-2.722559e-02,-1.081286e-02,2.859336e-02,8.497332e-03,7.321627e-04,4.111183e-02
1,1.151854e-03,2.840461e-02,1.366205e-01,-3.504661e-02,-1.649879e-01,-1.165797e-02,-9.156888e-02,8.724432e-02,-2.371246e-02,7.898096e-03,...,1.500122e-02,-1.922298e-02,1.193739e-02,1.310233e-02,-1.006083e-02,-2.148884e-02,2.914212e-02,-4.224745e-03,2.127501e-02,-1.879696e-02
2,5.402404e-03,1.134675e-02,5.411019e-02,-1.517680e-02,-2.377572e-02,-2.100344e-03,-1.433148e-02,-5.108403e-02,6.877048e-03,-4.172941e-03,...,-2.293278e-02,-3.296433e-02,6.455118e-02,2.225283e-02,2.900723e-02,-5.862264e-02,1.958000e-02,1.836610e-02,-2.579750e-02,9.988669e-03
3,7.655332e-04,1.117767e-02,4.116448e-02,-1.858101e-02,-8.013624e-03,-8.853561e-03,2.675275e-03,-1.035064e-02,-9.725729e-03,-3.357947e-04,...,4.598777e-04,-1.135351e-02,-1.059310e-02,-1.145976e-02,-5.422667e-03,5.171741e-03,-1.918576e-03,1.797697e-02,-3.193196e-03,-3.088185e-02
4,-1.765623e-17,-1.057163e-16,-7.103325e-17,7.504676e-16,-1.362751e-16,2.926488e-16,1.191594e-15,-9.121463e-16,8.085944e-17,1.170577e-16,...,-1.014482e-16,-3.554912e-16,-1.088508e-16,-8.939971e-17,7.013706e-17,-5.375870e-17,1.366482e-16,1.430117e-17,5.454044e-17,2.027219e-16


In [28]:
loadings = pd.DataFrame(SVD.components_,
                        index=component_names,
                        columns=term_df.columns).T

In [29]:
loadings['abs_component_1'] = np.abs(loadings.component_1)
loadings['abs_component_2'] = np.abs(loadings.component_2)

In [30]:
loadings.tail()

,component_1,component_2,component_3,component_4,component_5,component_6,component_7,component_8,component_9,component_10,...,component_93,component_94,component_95,component_96,component_97,component_98,component_99,component_100,abs_component_1,abs_component_2
youngest,0.000059,0.000759,0.003609,-0.001007,-0.001551,0.000772,-0.001531,-0.004259,-0.000328,-0.000280,...,-0.001377,0.008332,0.002738,-0.007588,-0.007640,0.009376,-0.006159,0.004307,0.000059,0.000759
youre,0.000044,0.001951,0.007738,-0.002801,0.001486,-0.000496,-0.000952,-0.000426,-0.003115,-0.001747,...,-0.008067,0.002748,-0.003928,0.008108,-0.002180,-0.002447,0.003258,0.001336,0.000044,0.001951
youth,0.000072,0.001339,0.006722,-0.001861,-0.001853,-0.000075,-0.004175,-0.001643,-0.004402,-0.000394,...,0.000892,-0.005739,-0.004191,0.007608,-0.005658,0.000559,-0.002687,0.005697,0.000072,0.001339
youtube,0.000087,0.035859,0.001394,-0.003475,-0.003233,0.005546,-0.000958,0.001902,-0.003657,-0.022571,...,0.021883,0.023125,0.040911,-0.001928,0.006975,-0.049203,-0.012039,0.026027,0.000087,0.035859
zealand,0.000093,0.002694,0.013557,-0.003463,-0.019439,-0.002408,-0.011043,0.018299,-0.004490,0.000655,...,0.017536,0.000839,0.019314,0.003079,0.011261,-0.000593,-0.009763,-0.015223,0.000093,0.002694


In [31]:
loadings.sort_values('abs_component_1',ascending=False).head(20)[['component_1']]

,component_1
worth,0.411728
biography,0.410734
net worth,0.407664
net,0.406981
biography net worth,0.404730
biography net,0.404730
neutrality,0.014333
net neutrality,0.014333
life,0.010919
bio biography,0.010459


In [32]:
loadings.sort_values('abs_component_2',ascending=False).head(20)[['component_2']]

,component_2
rokambola,0.492558
la,0.481783
repetirmastecno,0.352855
en,0.290822
el,0.253725
que,0.218619
para,0.208595
del,0.140477
los,0.123716
las,0.097853


# Cosine Similarity

In [33]:
cos_sim_mat = cosine_similarity(svd_matrix, svd_matrix)
cos_sim_mat.shape

(9956, 9956)

In [34]:
np.argsort(cos_sim_mat[1])[:-11:-1]

array([   1, 2679, 6860, 2169, 2656, 2138, 1739, 8851,  415, 3471])

In [35]:
np.sort(cos_sim_mat[1])[:-11:-1]

array([1.        , 0.73391359, 0.72786634, 0.72506317, 0.72499505,
       0.72360084, 0.72139795, 0.70876044, 0.70787806, 0.70559228])

In [36]:
df.loc[np.argsort(cos_sim_mat[50])[:-11:-1], :]

,headlines,news
50,Same Day Agra Tour,1
5717,DUBAI DESERT SAFARI EVENING DESERT SAFARI HUMM...,0
1303,This fart joke will brighten your day,1
6651,Arthritis pain Relieve symptoms of condition w...,0
921,Clear the Shelters day was a success here in T...,1
2235,The Shocking Day My Husband Intervened,1
3915,Prime Day is tomorrow,1
7970,Severe storms in Midwest as most of eastern US...,0
6311,6 cars and 6 unexpected Labor Day weekend geta...,0
7466,Americans Are Grilling More Steaks for Labor D...,0


# Train Test Splitting

In [37]:
X_train, X_test, y_train, y_test = train_test_split(svd_df, target, stratify=target)

In [38]:
knn = KNeighborsClassifier(n_neighbors=15)

In [39]:
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=15, p=2,
           weights='uniform')

In [40]:
knn.score(X_train, y_train)

0.7413954734163654

In [41]:
knn.score(X_test, y_test)

0.7095218963439133

# Grid Search

In [42]:
gs = GridSearchCV(KNeighborsClassifier(), {'n_neighbors': np.arange(3,10,5)}, cv=5)

In [43]:
gs.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_neighbors': array([3, 8])}, pre_dispatch='2*n_jobs',
       refit=True, scoring=None, verbose=0)

In [44]:
gs.best_params_

{'n_neighbors': 8}

In [45]:
gs.score(X_train, y_train)

0.7551895004687291

In [46]:
gs.score(X_test, y_test)

0.6922458818802733